In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

# import submitit

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
from IPython import get_ipython
import csv


findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
spark.sql("set spark.sql.execution.arrow.enabled=false")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 01:55:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 01:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/14 01:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/14 01:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/14 01:55:54 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


DataFrame[key: string, value: string]

In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data_part_repl.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

25/03/14 01:55:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df_population = spark.sql("SELECT * FROM population")

In [5]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [6]:
def whether_thisVIN_already_checked_for_part_replacement(thisVIN):
    path_to_VINs_data_part_repl="/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data_part_repl.csv"
    VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')
    if thisVIN in VINs_their_part_repl_is_checked['VIN'].values:
        print(f"VIN {thisVIN} found in the dataset.")
        return 1
    else:
        print(f"VIN {thisVIN} not found in the dataset.")
        return 0
    
        
def clean_csv(file_path, fixed_file_path, expected_number_of_columns):
    
    """
    Preprocess the CSV file to fix bad lines.

    Parameters:
    - file_path: Path to the input CSV file.
    - fixed_file_path: Path to save the fixed CSV file.
    - expected_columns: Expected number of columns in the CSV file.
    """
    with open(file_path, 'r', encoding='utf-8') as infile, open(fixed_file_path, 'w', encoding='utf-8') as outfile:
        buffer = ""
        line_count = 0
        skipped_lines = 0
        for line in infile:
            line_count += 1
            if not buffer:
                buffer = line.strip()
            else:
                buffer += ' ' + line.strip()
            
            # Check the number of columns
            columns_count = buffer.count(',') + 1
            if columns_count == expected_number_of_columns:
                outfile.write(buffer + '\n')
                buffer = ""
            elif columns_count > expected_number_of_columns:
                # Too many columns, reset the buffer
                # print(f"Skipping malformed line (line {line_count})")
                buffer = ""
                skipped_lines += 1
        
        # Write any remaining buffered line if it has the correct number of columns
        if buffer and buffer.count(',') + 1 == expected_number_of_columns:
            outfile.write(buffer + '\n')
        
        print(f"Total lines processed: {line_count}")
        print(f"Total lines skipped: {skipped_lines}")
        
# Function to submit a SLURM job and return the job ID
def submit_job(script_path, thisVIN, all_columns_names, account='vuh14_dibbs_sc', partition='sla-prio'):
    
    comma_delimited_string_for_all_columns_names = ','.join(all_columns_names)
    submit_command = f"sbatch --account={account} --partition={partition} {script_path} {thisVIN} {comma_delimited_string_for_all_columns_names}"
    print("The submit command for the submitted job is: ")
    # print(submit_command)
    result = subprocess.run(submit_command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        job_id = re.findall(r'\d+', result.stdout)[0]
        print(f"Job {job_id} submitted successfully.")
        return job_id
    else:
        print("Job submission failed.")
        return None
 

# Function to check the status of multiple jobs
def check_jobs_status(job_ids):
    jobs_running = set(job_ids)
    while jobs_running:
        for job_id in list(jobs_running):
            status_command = f"squeue -j {job_id}"
            result = subprocess.run(status_command, shell=True, capture_output=True, text=True)
            if not result.stdout:  # If the job is no longer in the queue
                print(f"Job {job_id} completed.")
                jobs_running.remove(job_id)
        if jobs_running:
            print("Some jobs are still running...")
            print(jobs_running)
            t.sleep(1)  # Check every 5 seconds

# Function to retrieve job outputs
def get_jobs_output(output_files):
    for output_file in output_files:
        try:
            with open(output_file, "r") as file:
                output = file.read()
            print(f"Output from {output_file}:")
            print(output)
        except FileNotFoundError:
            print(f"Output file {output_file} not found.")
        

In [7]:
# File paths
input_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv'
cleaned_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/cleaned_resultedData.csv'


all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features     


# Expected columns, based on your description (adjust this list as needed)
expected_columns = [f'col_{i}' for i in range(593)] + ['calendar_day'] + [f'col_{i}' for i in range(594, 721)] + \
                   ["if_parts_replaced_in_1th_15d", "if_parts_replaced_in_2nd_15d"]

expected_number_of_columns = 723 

clean_csv(input_file_path, cleaned_file_path, expected_number_of_columns)

Total lines processed: 1931596
Total lines skipped: 5634


In [8]:
# Load and clean the CSV
cleaned_resultedData = pd.read_csv(cleaned_file_path, header=None, names=all_columns_names, index_col=False, dtype='unicode')

cleaned_resultedData

,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920278,4V4NC9EH4GN956437,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1920279,4V4NC9EH4GN956437,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1920280,4V4NC9EH4GN956437,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1920281,4V4NC9EH4GN956437,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [ ]:
# Submitting multiple jobs
df_pop_VINs_sp = cleaned_resultedData['VIN'].unique()
# Convert the NumPy array to a Pandas DataFrame
df_pop_VINs = pd.DataFrame(df_pop_VINs_sp, columns=['VIN'])

job_ids = []
output_files = []

# Initialize a counter for submitted jobs
submitted_jobs_count = 0

for index, row in df_pop_VINs.iterrows():
    # Stop the loop if 10 jobs have already been submitted
    if submitted_jobs_count >= 10:
        print("Reached the maximum limit of 10 jobs.")
        break
    
    thisVIN = row['VIN']
    if whether_thisVIN_already_checked_for_part_replacement(thisVIN) == 1:
        print(f"This VIN = {thisVIN} has already been checked for the part replacement.")
        continue
    else:
        print(f"This VIN = {thisVIN} has not been checked for the part replacement.")
        script_path = f"/storage/home/yqf5148/work/volvoPennState/Jobs/submit_fixPartReplacement.sh"
        output_file = f"/storage/home/yqf5148/work/volvoPennState/Jobs/log_partReplacement/result_{thisVIN}.txt"
        job_id = submit_job(script_path, thisVIN, all_columns_names)

        if job_id:
            job_ids.append(job_id)
            output_files.append(output_file)
            submitted_jobs_count += 1  # Increment the counter after each job submission


# Check the status of all jobs 
check_jobs_status(job_ids)

# Retrieve and print outputs for all jobs
get_jobs_output(output_files)


/tmp/ipykernel_2300552/2520071988.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


VIN 4V4NC9EH6KN904736 not found in the dataset.
This VIN = 4V4NC9EH6KN904736 has not been checked for the part replacement.
The submit command for the submitted job is: 
Job 34987345 submitted successfully.
VIN 1M1AN4GY6LM018279 not found in the dataset.
This VIN = 1M1AN4GY6LM018279 has not been checked for the part replacement.
The submit command for the submitted job is: 


/tmp/ipykernel_2300552/2520071988.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


Job 34987351 submitted successfully.
VIN 4V4NC9EJ5KN210896 not found in the dataset.
This VIN = 4V4NC9EJ5KN210896 has not been checked for the part replacement.
The submit command for the submitted job is: 


/tmp/ipykernel_2300552/2520071988.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


...

Job 34987374 submitted successfully.
VIN 4V4NC9EJ2LN213451 not found in the dataset.
This VIN = 4V4NC9EJ2LN213451 has not been checked for the part replacement.
The submit command for the submitted job is: 


/tmp/ipykernel_2300552/2520071988.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


Job 34987376 submitted successfully.
VIN 4V4NC9EG3JN898643 not found in the dataset.
This VIN = 4V4NC9EG3JN898643 has not been checked for the part replacement.
The submit command for the submitted job is: 


/tmp/ipykernel_2300552/2520071988.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


Job 34987377 submitted successfully.
Reached the maximum limit of 10 jobs.
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are still running...
{'34987374', '34987376', '34987371', '34987377', '34987368', '34987351', '34987360', '34987367', '34987369', '34987345'}
Some jobs are